In [1]:
import psycopg2, geojson, json, folium, datetime, time, fiona, sklearn, os, utm
from sklearn import linear_model
from scipy.spatial.distance import cdist
from dateutil.parser import parse 
from sklearn import cross_validation
from sklearn import ensemble
from sklearn import grid_search
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.ensemble import AdaBoostRegressor 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error 
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble.forest import RandomForestRegressor
from shapely.geometry import Polygon, MultiPolygon, shape 
from pymongo import MongoClient 
from IPython.core.display import HTML
import pandas as pd
import numpy as np

ImportError: No module named 'psycopg2'

In [718]:
# load train and test data
train_raw = pd.read_csv('Taxi/Taxi_Train.csv', names=['trip id','departure time','arrival time','trip fare','number of passengers','lon of departure','lat of departure','lon of destination','lat of destination','TAZ of departure','TAZ of destination'])
test_raw = pd.read_csv('Taxi/Taxi_Query.csv', names=['trip id','departure time','arrival time','trip fare','number of passengers','lon of departure','lat of departure','lon of destination','lat of destination','TAZ of departure','TAZ of destination'])

In [719]:
# force data type conversion for columns
train_raw = train_raw.convert_objects(convert_dates='coerce', convert_numeric=True) 
test_raw = test_raw.convert_objects(convert_dates='coerce', convert_numeric=True) 

In [326]:
train_raw.head()

,trip id,departure time,arrival time,trip fare,number of passengers,lon of departure,lat of departure,lon of destination,lat of destination,TAZ of departure,TAZ of destination
0,0,2012-09-01 00:11:00,2012-09-01 00:20:00,13.20,1,-122.413540,37.802683,-122.421277,37.785395,38,30
1,1,2012-09-01 00:23:00,2012-09-01 00:31:00,10.65,1,-122.419700,37.786090,-122.435217,37.762177,30,94
2,2,2012-09-01 00:45:00,2012-09-01 00:49:00,9.00,1,-122.415120,37.774672,-122.407657,37.782615,10,11
3,3,2012-09-01 00:41:00,2012-09-01 00:54:00,13.95,2,-122.419392,37.806622,-122.415393,37.778115,40,10
4,4,2012-09-01 01:09:00,2012-09-01 01:13:00,7.35,1,-122.429722,37.797790,-122.418060,37.789032,45,32


In [6]:
train_labels = train_raw['trip fare']

In [7]:
train_features = train_raw[['number of passengers','TAZ of departure']]

# TASK 1

In [8]:
# creating feature with ride length in minutes
train_ride_len = train_raw.ix[:, 2] - train_raw.ix[:, 1]
train_ride_len_int = []
for i in train_ride_len:
    train_ride_len_int.append(int(i.minutes))
    
train_ride_len_int = pd.Series(train_ride_len_int)

In [ ]:
# adding ride length to train set
train_features['Trip length'] = train_ride_len_int

In [116]:
# converting 'TAZ departing zones' to strings
train_features['TAZ of departure'] = train_raw['TAZ of departure']

In [188]:
# initializing feature binarizer
binarizer = sklearn.preprocessing.LabelBinarizer()

In [454]:
# binarizing the 'TAZ of departure' features
try:
    del(train_tripstart_bin)
except:
    pass
train_tripstart_bin = binarizer.fit(train_raw['TAZ of departure'])
train_tripstart_bin = train_tripstart_bin.transform(train_raw['TAZ of departure'])
train_tripstart_bin = pd.DataFrame(train_tripstart_bin)
train_tripstart_bin.columns = train_tripstart_bin.columns.map(lambda x: 'start_' + str(x))

In [455]:
# binarizing the 'TAZ of departure' features
try:
    del(train_tripend_bin)
except:
    pass
train_tripend_bin = binarizer.fit(train_raw['TAZ of destination'])
train_tripend_bin = train_tripend_bin.transform(train_raw['TAZ of destination'])
train_tripend_bin = pd.DataFrame(train_tripend_bin)
train_tripend_bin.columns = train_tripend_bin.columns.map(lambda x: 'end_' + str(x))

In [151]:
# creating train_features2: added departure zone
train_features2 = pd.DataFrame(train_features['number of passengers']).join(train_tripstart_bin)
train_features2.shape

(50001, 227)

In [160]:
# creating train_features3: added end zone binary
train_features3 = train_features2.join(train_tripend_bin)
train_features3.shape

(50001, 808)

In [207]:
days_of_week = dict()
days_of_week = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}

In [208]:
# creating train_features4: adding day of week
train_features4 = pd.DataFrame(train_raw['departure time'])
train_features4['departure time'] = train_features4['departure time'].map(lambda x: days_of_week[x.dayofweek])

In [209]:
# binarizing days of week
train_daysofweek_bin = binarizer.fit(train_features4['departure time'])
train_daysofweek_bin = train_daysofweek_bin.transform(train_features4['departure time'])
train_daysofweek_bin = pd.DataFrame(train_daysofweek_bin)
train_daysofweek_bin.columns = train_daysofweek_bin.columns.map(lambda x: days_of_week[x])
train_features4 = pd.DataFrame(train_daysofweek_bin)

In [210]:
# feature 4 population
train_features4 = train_features4.join(train_features3)
train_features4.shape

(50001, 815)

In [268]:
# feature 5: add time of day
train_features5 = pd.DataFrame(train_raw['departure time'])
train_features5['departure time'] = train_features5['departure time'].map(lambda x: [str(x.hour)+'pm' if x.hour > 12 else str(x.hour)+'am'][0] )

In [269]:
# binarizing hour of day
train_hourofday_bin = binarizer.fit(train_features5['departure time'])
train_hourofday_bin = train_hourofday_bin.transform(train_features5['departure time'])
train_hourofday_bin = pd.DataFrame(train_hourofday_bin)
train_hourofday_bin.columns = train_hourofday_bin.columns.map(lambda x: [str(x)+'pm' if x > 12 else str(x)+'am'][0] )
train_features5 = train_hourofday_bin
train_features5.shape

(50001, 24)

In [354]:
# feature 5 population
train_features5 = train_features5.join(train_features4)
train_features5.shape

ValueError: columns overlap but no suffix specified: Index([u'Monday', u'Tuesday', u'Wednesday', u'Thursday', u'Friday', u'Saturday', u'Sunday', u'number of passengers', u'start_0', u'start_1', u'start_2', u'start_3', u'start_4', u'start_5', u'start_6', u'start_7', u'start_8', u'start_9', u'start_10', u'start_11', u'start_12', u'start_13', u'start_14', u'start_15', u'start_16', u'start_17', u'start_18', u'start_19', u'start_20', u'start_21', u'start_22', u'start_23', u'start_24', u'start_25', u'start_26', u'start_27', u'start_28', u'start_29', u'start_30', u'start_31', u'start_32', u'start_33', u'start_34', u'start_35', u'start_36', u'start_37', u'start_38', u'start_39', u'start_40', u'start_41', u'start_42', u'start_43', u'start_44', u'start_45', u'start_46', u'start_47', u'start_48', u'start_49', u'start_50', u'start_51', u'start_52', u'start_53', u'start_54', u'start_55', u'start_56', u'start_57', u'start_58', u'start_59', u'start_60', u'start_61', u'start_62', u'start_63', u'start_

In [671]:
# feature 6: add distance between start and end
#train_features6 = train_raw.ix[:,5:9]
start_coords = []
end_coords = []
distance = []

for a,b in zip(train_raw['lat of departure'],train_raw['lon of departure']):
    start_coords.append(utm.from_latlon(a,b)[0:2])
for a,b in zip(train_raw['lat of destination'],train_raw['lon of destination']):
    end_coords.append(utm.from_latlon(a,b)[0:2])
for a,z in zip(start_coords,end_coords):
    distance.append(cdist([a],[z])[0][0])
train_features6 = pd.DataFrame(pd.Series(distance),columns=['Distance'])
train_features6.head()

,Distance
0,2035.504963
1,2984.433776
2,1099.365428
3,3182.439496
4,1413.693347


In [672]:
train_features6 = train_features6.join(train_features5)

In [673]:
train_features6.shape

(50001, 840)

In [674]:
# train_features7: removing TAZ
train_features7 = train_features6.ix[:,:33]

In [675]:
# train_features8: removing no. of passengers
train_features8 = train_features7.ix[:,:-1]

## Testing Area

In [30]:
clf1 = LinearRegression()

In [559]:
regression_fitted = clf1.fit(train_features7,train_labels)

In [568]:
scores_lr = regression_fitted.predict(train_features7)

In [573]:
# scoring root mean squared error
round(round(mean_squared_error(train_labels,pd.Series(scores_lr)),2)**.5,4)

5.1556

In [562]:
rf1 = LinearRegression()
rf2 = sklearn.linear_model.Lasso()
rf3 = linear_model.Ridge()
rf4 = linear_model.ElasticNet()

print('10-fold cross validation:\n')

for rf, label in zip([rf1, rf2, clf3, rf4], ['Linear Regression', 'Lasso', 'Ridge', 'Elastic Net']):
    scores = cross_validation.cross_val_score(clf, train_features7.ix[:,:-1], train_labels, cv=10) 
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

10-fold cross validation:

Accuracy: 0.90 (+/- 0.02) [Linear Regression]
Accuracy: 0.90 (+/- 0.02) [Lasso]
Accuracy: 0.90 (+/- 0.02) [Ridge]
Accuracy: 0.90 (+/- 0.02) [Elastic Net]


In [566]:
# Random Forest
random_forest = RandomForestRegressor(random_state=0, n_estimators=50, max_depth=50)

In [567]:
# fitting
randforest_fitted = random_forest.fit(train_features7,train_labels)

In [572]:
# predicting
scores_rr = randforest_fitted.predict(train_features7)

In [574]:
# scoring root mean squared error
round(round(mean_squared_error(train_labels,pd.Series(scores_rr)),2)**.5,4)

3.8184

## Prepping Test Data

In [378]:
test_raw.head()

,trip id,departure time,arrival time,trip fare,number of passengers,lon of departure,lat of departure,lon of destination,lat of destination,TAZ of departure,TAZ of destination
0,50001,2012-09-17 15:22:00,-1,-1,1,-122.386792,37.617570,-122.392937,37.793532,239,0
1,50002,2012-09-16 22:08:00,-1,-1,1,-122.435182,37.762202,-122.438512,37.775893,94,85
2,50003,2012-09-16 19:49:00,-1,-1,1,-122.459563,37.790625,-122.410492,37.791650,0,29
3,50004,2012-09-16 15:53:00,-1,-1,1,-122.415940,37.800837,-122.431228,37.801518,39,44
4,50005,2012-09-16 15:20:00,-1,-1,1,-122.390272,37.615370,-122.417377,37.784307,239,9


In [ ]:
# initializing feature binarizer
binarizer = sklearn.preprocessing.LabelBinarizer()

In [379]:
# starting only with number of passangers 
test_feature = pd.DataFrame(test_raw['number of passengers'])

In [451]:
# adding binarized 'TAZ of departure' feature
try:
    del(train_tripstart_bin)
except:
    pass
test_tripstart_bin = binarizer.fit(test_raw['TAZ of departure'])
test_tripstart_bin = test_tripstart_bin.transform(train_raw['TAZ of departure'])
test_tripstart_bin = pd.DataFrame(test_tripstart_bin)
test_tripstart_bin.columns = test_tripstart_bin.columns.map(lambda x: 'start_' + str(x))
test_feature2 = test_feature.join(test_tripstart_bin)
test_feature2.head()

,number of passengers,start_0,start_1,start_2,start_3,start_4,start_5,start_6,start_7,start_8,...,start_176,start_177,start_178,start_179,start_180,start_181,start_182,start_183,start_184,start_185
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [390]:
# adding binarized 'TAZ of departure' feature
try:
    del(test_tripend_bin)
except:
    pass
test_tripend_bin = binarizer.fit(test_raw['TAZ of destination'])
test_tripend_bin = test_tripend_bin.transform(train_raw['TAZ of destination'])
test_tripend_bin = pd.DataFrame(test_tripend_bin)
test_tripend_bin.columns = test_tripend_bin.columns.map(lambda x: 'end_' + str(x))
test_feature3 = test_feature2.join(test_tripend_bin)
test_feature3.head()

,number of passengers,start_0,start_1,start_2,start_3,start_4,start_5,start_6,start_7,start_8,...,end_363,end_364,end_365,end_366,end_367,end_368,end_369,end_370,end_371,end_372
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [400]:
# creating train_features4: adding day of week
test_features4 = pd.DataFrame(test_raw['departure time'])
test_features4['departure time'] = test_features4['departure time'].map(lambda x: days_of_week[x.dayofweek])

In [401]:
# binarizing days of week
test_daysofweek_bin = binarizer.fit(test_features4['departure time'])
test_daysofweek_bin = test_daysofweek_bin.transform(test_features4['departure time'])
test_daysofweek_bin = pd.DataFrame(test_daysofweek_bin)
test_daysofweek_bin.columns = test_daysofweek_bin.columns.map(lambda x: days_of_week[x])
test_features4 = pd.DataFrame(test_daysofweek_bin)

In [402]:
# feature 4 population
test_features4 = test_features4.join(test_feature3)
test_features4.shape

(10000, 567)

In [404]:
# feature 5: add time of day
test_features5 = pd.DataFrame(test_raw['departure time'])
test_features5['departure time'] = test_features5['departure time'].map(lambda x: [str(x.hour)+'pm' if x.hour > 12 else str(x.hour)+'am'][0] )

In [405]:
# binarizing hour of day
test_hourofday_bin = binarizer.fit(test_features5['departure time'])
test_hourofday_bin = test_hourofday_bin.transform(test_features5['departure time'])
test_hourofday_bin = pd.DataFrame(test_hourofday_bin)
test_hourofday_bin.columns = test_hourofday_bin.columns.map(lambda x: [str(x)+'pm' if x > 12 else str(x)+'am'][0] )
test_features5 = test_hourofday_bin
test_hourofday_bin.shape

(10000, 24)

In [436]:
# feature 5 population
test_features5 = test_features5.join(test_features4)
test_features5.shape

(10000, 591)

In [690]:
# feature 6: add distance between start and end
#train_features6 = train_raw.ix[:,5:9]
start_coords = []
end_coords = []
distance = []

for a,b in zip(test_raw['lat of departure'],test_raw['lon of departure']):
    start_coords.append(utm.from_latlon(a,b)[0:2])
for a,b in zip(test_raw['lat of destination'],test_raw['lon of destination']):
    end_coords.append(utm.from_latlon(a,b)[0:2])
for a,z in zip(start_coords,end_coords):
    distance.append(cdist([a],[z])[0][0])
test_features6 = pd.DataFrame(pd.Series(distance),columns=['Distance'])
test_features6.head()

,Distance
0,19530.579939
1,1547.085620
2,4322.121271
3,1348.023220
4,18895.287592


In [691]:
test_features6 = test_features6.join(test_features5)

In [692]:
test_features6.shape

(10000, 592)

In [693]:
# test_features7: removing TAZ
test_features7 = test_features6.ix[:,:33]

### Matching the dimensions for both test and train feature sets

In [464]:
zones = [test_tripstart_bin.columns, train_tripstart_bin.columns, test_tripend_bin.columns, train_tripend_bin.columns]

In [694]:
# tracking missing start-columns in test that are present in train
missing_start_zones = []
for i in zones[1]:
    if i not in zones[0]: missing_start_zones.append(i)
len(missing_start_zones)

40

In [695]:
# adding missing start-columns in test that are present in train
zeroes = np.zeros(test_features6.shape[0])
for i in missing_start_zones:
    test_features6 = test_features6.join(pd.DataFrame(zeroes,columns=[i]))
test_features6.shape

(10000, 632)

In [696]:
# tracking missing end-columns in test that are present in train
missing_end_zones = []
for i in zones[3]:
    if i not in zones[2]: missing_end_zones.append(i)
len(missing_end_zones)

208

In [697]:
# adding missing end-columns in test that are present in train
zeroes = np.zeros(test_features6.shape[0])
for i in missing_end_zones:
    test_features6 = test_features6.join(pd.DataFrame(zeroes,columns=[i]))
test_features6.shape

(10000, 840)

In [698]:
# verifying that train and test have same number of columns now
print(test_features6.shape,train_features6.shape)

((10000, 840), (50001, 840))


In [623]:
test_features7.columns

Index([u'direct_distance', u'0am', u'1am', u'2am', u'3am', u'4am', u'5am', u'6am', u'7am', u'8am', u'9am', u'10am', u'11am', u'12am', u'13pm', u'14pm', u'15pm', u'16pm', u'17pm', u'18pm', u'19pm', u'20pm', u'21pm', u'22pm', u'23pm', u'Monday', u'Tuesday', u'Wednesday', u'Thursday', u'Friday', u'Saturday', u'Sunday', u'number of passengers'], dtype='object')

## Generating prediction from Test set (Taxy_Query.csv)

In [582]:
# automate creation of csv for kaggle submission
def create_submission(filename,scores):
    df_submission = pd.DataFrame([test_raw['trip id'].astype(str),scores.Fare]).T
    pd.DataFrame.to_csv(pd.DataFrame([test_raw['trip id'].astype(str),scores.Fare]).T, path_or_buf=filename, header=['Id','Fare'], index=False)
    
    print(df_submission.shape)
    df_submission.head()

In [709]:
# automate crossvalidation and printing of results
def crossvalidation(func, arr_train_feature, predictions_train):
    scores = cross_validation.cross_val_score(func, arr_train_feature, predictions_train, cv=5, scoring='mean_squared_error')
    print("Accuracy: %0.4f (+/- %0.2f)" % ((scores.mean() ** 2) ** 0.25, scores.std() * 2))
#    return scores

In [699]:
# fitting linear regression
regression_fitted = clf1.fit(train_features6,train_labels)

In [700]:
# linear regresion predictions
scores_lr = regression_fitted.predict(test_features6)
scores_lr = pd.DataFrame(scores_lr)
scores_lr.columns = ['Fare']

In [686]:
# crossvalidatoion on linear regression 
crossvalidation(clf1,train_features7,train_labels)

Accuracy: 5.3019 (+/- 6.99)


In [701]:
# crossvalidatoion on linear regression 
crossvalidation(clf1,train_features6,train_labels)

Accuracy: 5.0638 (+/- 5.75)


In [688]:
# crossvalidatoion on linear regression 
crossvalidation(clf1,train_features8,train_labels)

Accuracy: 5.3019 (+/- 6.99)


In [702]:
# creating submission from linear regression
scores_lr.Fare.map(lambda x: [x * -1 if x < 0][0])
create_submission('submission4.csv',scores_lr)

(10000, 2)


In [ ]:
# initiating Random Forest Regressor
random_forest = RandomForestRegressor(random_state=0, n_estimators=50, max_depth=50)

In [628]:
# fitting random forest regression
randforest_fitted = random_forest.fit(train_features6,train_labels)

In [629]:
# random forest regresion predictions
scores_rr = randforest_fitted.predict(test_features6)
scores_rr = pd.DataFrame(scores_rr)
scores_rr.columns = ['Fare']
scores_rr.shape

(10000, 1)

In [630]:
# scoring root mean squared error wtih random forest
round(round(mean_squared_error(train_labels,pd.Series(randforest_fitted.predict(train_features6))),2)**.5,4)

2.3937

In [683]:
# crossvalidatoion on regression 
cv_scores = crossvalidation(random_forest,train_features7,train_labels)

Accuracy: 5.58 (+/- 8.17)


In [631]:
# creating submission from linear regression
create_submission('submission3.csv',scores_rr)

(10000, 2)


In [705]:
# initianig GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0, loss='ls')

In [710]:
# gbr crossvalidation train_features7
crossvalidation(gbr,train_features7, train_labels)

Accuracy: 5.1539 (+/- 5.80)


In [715]:
# gbr crossvalidation train_features6
crossvalidation(gbr,train_features6, train_labels)

Accuracy: 5.0943 (+/- 5.25)


In [713]:
# gradiant boosting regressor predictions
gbr_fitted = gbr.fit(train_features7,train_labels)
scores_gbr = gbr_fitted.predict(test_features7)
scores_gbr = pd.DataFrame(scores_rr)
scores_gbr.columns = ['Fare']
scores_gbr.shape

(10000, 1)

In [714]:
# creating submission from gradiant boosting regressor
create_submission('submission5.csv',scores_gbr)

(10000, 2)


## Using data from pgrouting features

In [738]:
train_raw_routed = pd.read_csv('Taxi_Data_Routed/Taxi_Train_Routed.csv')
test_raw_routed = pd.read_csv('Taxi_Data_Routed/Taxi_Query_Routed.csv')

In [731]:
train_raw_routed.shape

(50001, 14)

In [733]:
# Creating train_features9
train_features9 = train_raw_routed.ix[:,-3:]

In [754]:
# Creating test_features9
test_features9 = test_raw_routed.ix[:,-3:]

In [737]:
# crossvalidatoion on linear regression 
crossvalidation(clf1,train_features9,train_labels)

Accuracy: 4.9285 (+/- 5.41)


In [745]:
# Creating train_features10: train_features9 plus times of day
train_features10 = train_features9.join(train_features7.ix[:,1:25])

In [756]:
# Creating test_features10: test_features9 plus times of day
test_features10 = test_features9.join(test_features7.ix[:,1:25])

In [747]:
# crossvalidatoion on linear regression 
crossvalidation(clf1,train_features10,train_labels)

Accuracy: 4.8941 (+/- 5.36)


In [751]:
# Creating train_features11: train_features10 plus days of week
train_features11 = train_features10.join(train_features7.ix[:,25:-1])

In [758]:
# Creating test_features11: test_features10 plus days of week
test_features11 = test_features10.join(test_features7.ix[:,25:-1])

In [773]:
# crossvalidatoion on linear regression 
crossvalidation(clf1,train_features11,train_labels)

Accuracy: 4.8939 (+/- 5.37)


In [759]:
regression_fitted = clf1.fit(train_features11,train_labels)
scores_lr = regression_fitted.predict(test_features11)
scores_lr = pd.DataFrame(scores_lr)
scores_lr.columns = ['Fare']

In [760]:
# creating submission from gradiant boosting regressor
create_submission('submission6.csv',scores_lr)

(10000, 2)


In [775]:
# Creating train_features12: test_features11 plus number of passengers
train_features12 = pd.DataFrame(train_raw.ix[:,4]).join(train_features11)
train_features12.shape

(50001, 35)

In [794]:
# Creating test_features12: test_features11 plus number of passengers
test_features12 = pd.DataFrame(test_raw.ix[:,4]).join(test_features11)
test_features12.shape

(10000, 35)

In [779]:
# crossvalidatoion on linear regression 
crossvalidation(clf1,train_features12.ix[:,:-7],train_labels)

Accuracy: 4.8939 (+/- 5.36)


In [777]:
train_features12.ix[:,:-7].head()

,number of passengers,dist,timecost,distcost,0am,1am,2am,3am,4am,5am,...,21pm,22pm,23pm,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,1,0.018940,0.081062,0.029604,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,0.028506,0.116872,0.036868,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,0.010899,0.008682,0.007640,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,2,0.028786,0.086223,0.030821,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,0.014584,0.041175,0.017230,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [780]:
# crossvalidatoion on gbr 
crossvalidation(gbr,train_features12,train_labels)

Accuracy: 4.8715 (+/- 4.98)


In [795]:
gbr_fitted = gbr.fit(train_features12,train_labels)
scores_gbr = gbr_fitted.predict(test_features12)
scores_gbr = pd.DataFrame(scores_gbr)
scores_gbr.columns = ['Fare']

In [796]:
test_features12.shape

(10000, 35)

In [797]:
# creating submission from gradiant boosting regressor
create_submission('submission7.csv',scores_gbr)

(10000, 2)


## Testing parameter exhaustion with GridSearchCV and GBR

In [832]:
parameters = {'min_samples_split':[2, 3, 4], 'learning_rate':[0.05]} 

In [833]:
clf = grid_search.GridSearchCV(ensemble.GradientBoostingRegressor(), parameters)

In [834]:
clf_fitted = clf.fit(train_features12,train_labels)

In [835]:
clf_fitted.best_estimator_

GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.05, loss='ls',
             max_depth=3, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=3, n_estimators=100,
             random_state=None, subsample=1.0, verbose=0, warm_start=False)

In [837]:
# initianig Optimized GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.05, max_depth=3, min_samples_split=3, random_state=0, loss='ls')

In [838]:
# crossvalidatoion on gbr 
crossvalidation(gbr,train_features12,train_labels)

Accuracy: 4.8196 (+/- 5.50)


In [840]:
# gradiant boosting regressor predictions
gbr_fitted = gbr.fit(train_features12,train_labels)
scores_gbr = gbr_fitted.predict(test_features12)
scores_gbr = pd.DataFrame(scores_gbr)
scores_gbr.columns = ['Fare']
scores_gbr.shape

(10000, 1)

In [841]:
# creating submission from grid search
create_submission('submission9.csv',scores_gscv)

(10000, 2)


In [844]:
os.system("say 'process complete'")

0

# TASK 2

In [1200]:
from sklearn import cluster
from time import time
import shapely, fiona
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from dateutil.parser import parse 

In [846]:
def inline_map(m, width=650, height=500):
    """Takes a folium instance and embed HTML."""
    m._build_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{}" '
                 'style="width: {}px; height: {}px; '
                 'border: none"></iframe>'.format(srcdoc, width, height))
    return embed

In [1167]:
base_time = np.datetime64('2012-09-02 20:00:00')

In [1161]:
train_raw.ix[:1,:]

,trip id,departure time,arrival time,trip fare,number of passengers,lon of departure,lat of departure,lon of destination,lat of destination,TAZ of departure,TAZ of destination
0,0,2012-09-01 00:11:00,2012-09-01 00:20:00,13.20,1,-122.41354,37.802683,-122.421277,37.785395,38,30
1,1,2012-09-01 00:23:00,2012-09-01 00:31:00,10.65,1,-122.41970,37.786090,-122.435217,37.762177,30,94


In [1178]:
base_df = pd.DataFrame(train_raw[train_raw['departure time'] > base_time])

In [1179]:
base_df[base_df['departure time'] < base_time + 1]

,trip id,departure time,arrival time,trip fare,number of passengers,lon of departure,lat of departure,lon of destination,lat of destination,TAZ of departure,TAZ of destination


In [1146]:
train_raw.ix[:2,1]

0   2012-09-01 00:11:00
1   2012-09-01 00:23:00
2   2012-09-01 00:45:00
Name: departure time, dtype: datetime64[ns]

In [988]:
coords_start = zip(train_raw.ix[:,6], train_raw.ix[:,5])
coords_end = zip(train_raw.ix[:,8],train_raw.ix[:,7])

In [964]:
k_cluster_miniBatch_start = []
results_cluster_miniBatch = []
for n in range(20,61,20):
    some_mb = cluster.MiniBatchKMeans(n_clusters=n)
    timer = time()
    some_mb.fit(coords_start)
    timer = time() - timer
    results_cluster_miniBatch.append(timer)
    k_cluster_miniBatch_start.append(some_mb)
os.system('say "process complete!"');

In [965]:
clustering_start1 = k_cluster_miniBatch_start[0]
clustering_start2 = k_cluster_miniBatch_start[1]
clustering_start3 = k_cluster_miniBatch_start[2]

In [989]:
k_cluster_miniBatch_end = []
results_cluster_miniBatch = []
for n in range(20,61,20):
    some_mb = cluster.MiniBatchKMeans(n_clusters=n)
    timer = time()
    some_mb.fit(coords_end)
    timer = time() - timer
    results_cluster_miniBatch.append(timer)
    k_cluster_miniBatch_end.append(some_mb)
os.system('say "process complete!"');

In [990]:
clustering_end1 = k_cluster_miniBatch_end[0]
clustering_end2 = k_cluster_miniBatch_end[1]
clustering_end3 = k_cluster_miniBatch_end[2]

In [1119]:
#dict(zip(pd.Series(clustering_start2.labels_).value_counts().index,pd.Series(clustering_start2.labels_).value_counts()))
start_counts = pd.Series(clustering_start2.labels_).value_counts()
end_counts = pd.Series(clustering_end2.labels_).value_counts()

In [1121]:
start_counts_index = start_counts.index
end_counts_index = end_counts.index
start_counts = [ float(x) for x in start_counts ]
end_counts = [ float(x) for x in end_counts ]

In [1122]:
start_counts = dict(zip(start_counts_index,start_counts))
end_counts = dict(zip(end_counts_index,end_counts))

In [1095]:
normalizer = MinMaxScaler()

In [1124]:
start_normalized = normalizer.fit_transform(start_counts.values())
end_normalized = normalizer.fit_transform(end_counts.values())

In [1384]:
# testing base map
width, height = 750, 550
taxi_loc = folium.Map(location=[37.759953, -122.44147], zoom_start=12,
                    tiles='OpenStreetMap', width=width, height=height)

for c,e in zip(clustering_end2.cluster_centers_, end_normalized):
    taxi_loc.circle_marker(location=(c[0],c[1]), radius=(300 * (e + 0.3)), line_color=None, fill_color='red', fill_opacity=0.5)

for c,s in zip(clustering_start2.cluster_centers_, start_normalized):
    taxi_loc.circle_marker(location=(c[0],c[1]), radius=(300 * (s + 0.3)), line_color=None, fill_color='white', fill_opacity=0.9)

taxi_loc.geo_json(geo_path='TAZ.geojson', fill_opacity=0.2, line_opacity=0.2)
    
inline_map(taxi_loc, width, height)

In [1006]:
# opening shape file and loading collection into variable
shp = fiona.open('mtc_taz1454_clipurb2002/mtc_taz1454_clipurb2002.shp','r')

In [1088]:
for n,i in enumerate(poly_test):
    poly_test[n] = tuple(i)

In [1179]:
train_features5['departure time'] = train_features5['departure time'].map(lambda x: [str(x.hour)+'pm' if x.hour > 12 else str(x.hour)+'am'][0] )

In [1250]:
train_taz = train_features6.ix[:,33:]
test_taz = test_features6.ix[:,33:]
print(train_taz.shape, test_taz.shape)

((50001, 807), (10000, 807))


In [1251]:
# creating training features 13:  same as train_features 12 plus the binarized zones
train_features13 = train_features12.join(train_taz)
test_features13 = test_features12.join(test_taz)
print(train_features13.shape, test_features13.shape)

((50001, 842), (10000, 842))


In [1254]:
# gradiant boosting regressor predictions
gbr_fitted = gbr.fit(train_features13,train_labels)
scores_gbr = gbr_fitted.predict(test_features13)
scores_gbr = pd.DataFrame(scores_gbr)
scores_gbr.columns = ['Fare']
scores_gbr.shape

(10000, 1)

In [1252]:
def get_similar(date):
    date = parse(date)
    
    hour = date.hour
    if hour > 12:
        hour = str(hour) + 'pm'
    else:
        hour = str(hour) + 'am'
        
    weekday = days_of_week[date.weekday()]
    
    return train_features13[(train_features13[weekday] == 1) & (train_features13[hour] == 1)]

In [1358]:
label_start, label_end = [], []
for i in output.iterrows():
    for n,p in enumerate(zip(i[1][35:261].index,i[1][35:261])):
        if p[1] == 1:
            label_start.append(p[0])
    for n,p in enumerate(zip(i[1][261:].index,i[1][261:])):
        if p[1] == 1:
            label_end.append(p[0])
label_start = pd.Series(label_start)
label_end = pd.Series(label_end)

In [1483]:
def cluster_measure(output):
    df = train_raw.ix[output.index]
    some_mb = cluster.MiniBatchKMeans(n_clusters=[5 if df.shape[0] < 10 else 10][0])
    some_mb.fit(zip(df['lat of departure'],df['lon of departure']))

    df = pd.DataFrame([some_mb.labels_,df['trip fare']], index=['label','fare']).T
    counts = dict(zip(set(some_mb.labels_), np.zeros(len(set(some_mb.labels_)))))

    for i in counts:
        counts[i] = df[df['label'] == i]['fare'].mean() * df[df['label'] == i]['fare'].count()

    counts_normalized = normalizer.fit_transform(counts.values())
    counts = dict(zip(counts, counts_normalized))
    return some_mb, counts
   

In [1486]:
output = get_similar('09/08/2015 20:00:00')
some_mb, counts = cluster_measure(output)

In [1487]:
 # testing base map
width, height = 750, 550
taxi_loc = folium.Map(location=[37.759953, -122.44147], zoom_start=12,
                    tiles='OpenStreetMap', width=width, height=height)

for c,s in zip(some_mb.cluster_centers_, counts):
    taxi_loc.circle_marker(location=(c[0],c[1]), radius=(600 * (counts[s] + 0.3)), line_color=None, fill_color='black', fill_opacity=0.7)

taxi_loc.geo_json(geo_path='TAZ.geojson', fill_opacity=0.2, line_opacity=0.3)

inline_map(taxi_loc, width, height)